In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!pip install sentencepiece
!git clone https://github.com/renatoviolin/xlnet
!wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
!unzip cased_L-24_H-1024_A-16.zip
!mv xlnet_cased_L-24_H-1024_A-16 xlnet/
!mkdir -p xlnet/SQUAD_DATA
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
!mv train-v2.0.json xlnet/SQUAD_DATA/
!mv dev-v2.0.json xlnet/SQUAD_DATA/
!mkdir -p xlnet/SQUAD_OUTPUT

     |████████████████████████████████| 1.0MB 3.4MB/s 
Cloning into 'xlnet'...
remote: Enumerating objects: 126, done.
remote: Total 126 (delta 0), reused 0 (delta 0), pack-reused 126
Receiving objects: 100% (126/126), 133.51 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (66/66), done.
--2019-07-15 03:18:19--  https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c11::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1338042341 (1.2G) [application/zip]
Saving to: ‘cased_L-24_H-1024_A-16.zip’

cased_L-24_H-1024_A 100%[===================>]   1.25G   187MB/s    in 6.2s    

2019-07-15 03:18:25 (207 MB/s) - ‘cased_L-24_H-1024_A-16.zip’ saved [1338042341/1338042341]

Archive:  cased_L-24_H-1024_A-16.zip
   creating: xlnet_cased_L-24_H-1024_A-16/
  inflating: xlnet_case

In [0]:
# coding=utf-8
import sys
sys.path.insert(0,"xlnet/")


from absl import flags
import absl.logging as _logging  # pylint: disable=unused-import

import collections
import os
import time
import math
import json
import six
import random
import gc

import numpy as np

if six.PY2:
  import cPickle as pickle
else:
  import pickle

import tensorflow as tf
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids, encode_pieces, printable_text
import function_builder
import model_utils
import squad_utils
from data_utils import SEP_ID, CLS_ID, VOCAB_SIZE

import json
import time

SPIECE_UNDERLINE = u'▁'

SEG_ID_P   = 0
SEG_ID_Q   = 1
SEG_ID_CLS = 2
SEG_ID_PAD = 3


In [0]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

In [0]:
# Preprocessing
flags.DEFINE_bool("do_prepro", default=False,
      help="Perform preprocessing only.")
flags.DEFINE_integer("num_proc", default=1,
      help="Number of preprocessing processes.")
flags.DEFINE_integer("proc_id", default=0,
      help="Process id for preprocessing.")

# Model
flags.DEFINE_string("model_config_path", default='xlnet_cased_L-24_H-1024_A-16/xlnet_config.json',
      help="Model config path.")
flags.DEFINE_float("dropout", default=0.1,
      help="Dropout rate.")
flags.DEFINE_float("dropatt", default=0.1,
      help="Attention dropout rate.")
flags.DEFINE_integer("clamp_len", default=-1,
      help="Clamp length.")
flags.DEFINE_string("summary_type", default="last",
      help="Method used to summarize a sequence into a vector.")
flags.DEFINE_bool("use_bfloat16", default=False,
      help="Whether to use bfloat16.")

# Parameter initialization
flags.DEFINE_enum("init", default="normal",
                  enum_values=["normal", "uniform"],
                  help="Initialization method.")
flags.DEFINE_float("init_std", default=0.02,
                   help="Initialization std when init is normal.")
flags.DEFINE_float("init_range", default=0.1,
                   help="Initialization std when init is uniform.")


# I/O paths
flags.DEFINE_bool("overwrite_data", default=False,
                  help="If False, will use cached data if available.")
flags.DEFINE_string("init_checkpoint", default='xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt',
                    help="checkpoint path for initializing the model. "
                    "Could be a pretrained model or a finetuned model.")
flags.DEFINE_bool("init_global_vars", default=False,
                  help="If true, init all global vars. If false, init "
                  "trainable vars only.")
flags.DEFINE_string("output_dir", default="",
                    help="Output dir for TF records.")
flags.DEFINE_string("predict_dir", default="",
                    help="Dir for predictions.")
flags.DEFINE_string("spiece_model_file", default="xlnet_cased_L-24_H-1024_A-16/spiece.model",
                    help="Sentence Piece model path.")
flags.DEFINE_string("model_dir", default="/content/drive/My Drive/colab_data/SQUAD_MODEL",
                    help="Directory for saving the finetuned model.")
flags.DEFINE_string("train_file", default="",
                    help="Path of train file.")
flags.DEFINE_string("predict_file", default="qna.json",
                    help="Path of prediction file.")

# Data preprocessing config
flags.DEFINE_integer("max_seq_length",
                     default=512, help="Max sequence length")
flags.DEFINE_integer("max_query_length",
                     default=64, help="Max query length")
flags.DEFINE_integer("doc_stride",
                     default=128, help="Doc stride")
flags.DEFINE_integer("max_answer_length",
                     default=64, help="Max answer length")
flags.DEFINE_bool("uncased", default=False, help="Use uncased data.")

# TPUs and machines
flags.DEFINE_bool("use_tpu", default=False, help="whether to use TPU.")
flags.DEFINE_integer("num_hosts", default=1, help="How many TPU hosts.")
flags.DEFINE_integer("num_core_per_host", default=8,
      help="8 for TPU v2 and v3-8, 16 for larger TPU v3 pod. In the context "
      "of GPU training, it refers to the number of GPUs used.")
flags.DEFINE_string("tpu_job_name", default=None, help="TPU worker job name.")
flags.DEFINE_string("tpu", default=None, help="TPU name.")
flags.DEFINE_string("tpu_zone", default=None, help="TPU zone.")
flags.DEFINE_string("gcp_project", default=None, help="gcp project.")
flags.DEFINE_string("master", default=None, help="master")
flags.DEFINE_integer("iterations", default=1000,
                     help="number of iterations per TPU training loop.")

# Training
flags.DEFINE_bool("do_train", default=True, help="whether to do training")
flags.DEFINE_integer("train_batch_size", default=48,
                     help="batch size for training")
flags.DEFINE_integer("train_steps", default=8000,
                     help="Number of training steps")
flags.DEFINE_integer("warmup_steps", default=0, help="number of warmup steps")
flags.DEFINE_integer("save_steps", default=None,
                     help="Save the model for every save_steps. "
                     "If None, not to save any model.")
flags.DEFINE_integer("max_save", default=5,
                     help="Max number of checkpoints to save. "
                     "Use 0 to save all.")
flags.DEFINE_integer("shuffle_buffer", default=2048,
                     help="Buffer size used for shuffle.")

# Optimization
flags.DEFINE_float("learning_rate", default=3e-5, help="initial learning rate")
flags.DEFINE_float("min_lr_ratio", default=0.0,
                   help="min lr ratio for cos decay.")
flags.DEFINE_float("clip", default=1.0, help="Gradient clipping")
flags.DEFINE_float("weight_decay", default=0.00, help="Weight decay rate")
flags.DEFINE_float("adam_epsilon", default=1e-6, help="Adam epsilon")
flags.DEFINE_string("decay_method", default="poly", help="poly or cos")
flags.DEFINE_float("lr_layer_decay_rate", default=0.75,
                   help="Top layer: lr[L] = FLAGS.learning_rate."
                   "Lower layers: lr[l-1] = lr[l] * lr_layer_decay_rate.")

# Eval / Prediction
flags.DEFINE_bool("do_predict", default=False, help="whether to do predict")
flags.DEFINE_integer("predict_batch_size", default=32,
                     help="batch size for prediction")
flags.DEFINE_integer("n_best_size", default=5,
                     help="n best size for predictions")
flags.DEFINE_integer("start_n_top", default=5, help="Beam size for span start.")
flags.DEFINE_integer("end_n_top", default=5, help="Beam size for span end.")
flags.DEFINE_string("target_eval_key", default="best_f1",
                    help="Use has_ans_f1 for Model I.")


FLAGS = flags.FLAGS

In [0]:
RawResult = collections.namedtuple("RawResult",
    ["unique_id", "start_top_log_probs", "start_top_index",
    "end_top_log_probs", "end_top_index", "cls_logits"])

_PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "PrelimPrediction",
    ["feature_index", "start_index", "end_index",
    "start_log_prob", "end_log_prob"])

_NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "NbestPrediction", ["text", "start_log_prob", "end_log_prob"])


class SquadExample(object):
  """A single training/test example for simple sequence classification.

     For examples without an answer, the start and end position are -1.
  """

  def __init__(self,
               qas_id,
               question_text,
               paragraph_text,
               orig_answer_text=None,
               start_position=None,
               is_impossible=False):
    self.qas_id = qas_id
    self.question_text = question_text
    self.paragraph_text = paragraph_text
    self.orig_answer_text = orig_answer_text
    self.start_position = start_position
    self.is_impossible = is_impossible

  def __str__(self):
    return self.__repr__()

  def __repr__(self):
    s = ""
    s += "qas_id: %s" % (printable_text(self.qas_id))
    s += ", question_text: %s" % (
        printable_text(self.question_text))
    s += ", paragraph_text: [%s]" % (" ".join(self.paragraph_text))
    if self.start_position:
      s += ", start_position: %d" % (self.start_position)
    if self.start_position:
      s += ", is_impossible: %r" % (self.is_impossible)
    return s


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               unique_id,
               example_index,
               doc_span_index,
               tok_start_to_orig_index,
               tok_end_to_orig_index,
               token_is_max_context,
               input_ids,
               input_mask,
               p_mask,
               segment_ids,
               paragraph_len,
               cls_index,
               start_position=None,
               end_position=None,
               is_impossible=None):
    self.unique_id = unique_id
    self.example_index = example_index
    self.doc_span_index = doc_span_index
    self.tok_start_to_orig_index = tok_start_to_orig_index
    self.tok_end_to_orig_index = tok_end_to_orig_index
    self.token_is_max_context = token_is_max_context
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.p_mask = p_mask
    self.segment_ids = segment_ids
    self.paragraph_len = paragraph_len
    self.cls_index = cls_index
    self.start_position = start_position
    self.end_position = end_position
    self.is_impossible = is_impossible

    
class FeatureWriter(object):
  """Writes InputFeature to TF example file."""

  def __init__(self, filename, is_training):
    self.filename = filename
    self.is_training = is_training
    self.num_features = 0
    self._writer = tf.python_io.TFRecordWriter(filename)

  def process_feature(self, feature):
    """Write a InputFeature to the TFRecordWriter as a tf.train.Example."""
    self.num_features += 1

    def create_int_feature(values):
      feature = tf.train.Feature(
          int64_list=tf.train.Int64List(value=list(values)))
      return feature

    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["unique_ids"] = create_int_feature([feature.unique_id])
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_float_feature(feature.input_mask)
    features["p_mask"] = create_float_feature(feature.p_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)

    features["cls_index"] = create_int_feature([feature.cls_index])

    if self.is_training:
      features["start_positions"] = create_int_feature([feature.start_position])
      features["end_positions"] = create_int_feature([feature.end_position])
      impossible = 0
      if feature.is_impossible:
        impossible = 1
      features["is_impossible"] = create_float_feature([impossible])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    self._writer.write(tf_example.SerializeToString())

  def close(self):
    self._writer.close()


In [0]:
def create_json(paragraph, ques):
    para_dic = {}
    context_count = 0
    para_dic["paragraphs"] = []
    for i, context in enumerate(paragraph):
        para_dic['paragraphs'].append({})
        para_dic['paragraphs'][context_count]['context'] = context
        qas_count = 0
        para_dic['paragraphs'][context_count]['qas'] = []
        for q in [ques[i]]:
            para_dic['paragraphs'][context_count]['qas'].append({})
            para_dic['paragraphs'][context_count]['qas'][qas_count]['answers'] = []
            para_dic['paragraphs'][context_count]['qas'][qas_count]['answers'].append({})
            para_dic['paragraphs'][context_count]['qas'][qas_count]['answers'][0]['answer_start'] = 0
            para_dic['paragraphs'][context_count]['qas'][qas_count]['answers'][0]['text'] = ""
            para_dic['paragraphs'][context_count]['qas'][qas_count]['question']= q
            para_dic['paragraphs'][context_count]['qas'][qas_count]['id'] = str(context_count) + str(qas_count)
            qas_count += 1
        context_count += 1
    data = {"data":[para_dic]}
    return json.dumps(data)


In [0]:
def get_answer(paragraph, question):
    qna_json = create_json([paragraph], [question])
    start = time.time()
    all_predictions_json , all_nbest_json =find_answer(qna_json=qna_json)
    end = time.time()
    N = 3
    result = []
    for row in all_nbest_json['00'][:N]:
        result.append({'text':row['text'],'probability':row['probability']})


    print('Time taken ', end-start , ' seconds')
    return result


In [0]:
def read_squad_examples(qna_json,is_training=False):
  """Read a SQuAD json file into a list of SquadExample."""
  input_data = json.loads(qna_json)["data"]

  examples = []
  for entry in input_data:
    for paragraph in entry["paragraphs"]:
      paragraph_text = paragraph["context"]

      for qa in paragraph["qas"]:
        qas_id = qa["id"]
        question_text = qa["question"]
        start_position = None
        orig_answer_text = None
        is_impossible = False

        if is_training:
          is_impossible = qa["is_impossible"]
          if (len(qa["answers"]) != 1) and (not is_impossible):
            raise ValueError(
                "For training, each question should have exactly 1 answer.")
          if not is_impossible:
            answer = qa["answers"][0]
            orig_answer_text = answer["text"]
            start_position = answer["answer_start"]
          else:
            start_position = -1
            orig_answer_text = ""

        example = SquadExample(
            qas_id=qas_id,
            question_text=question_text,
            paragraph_text=paragraph_text,
            orig_answer_text=orig_answer_text,
            start_position=start_position,
            is_impossible=is_impossible)
        examples.append(example)

  return examples


def _convert_index(index, pos, M=None, is_start=True):
  if index[pos] is not None:
    return index[pos]
  N = len(index)
  rear = pos
  while rear < N - 1 and index[rear] is None:
    rear += 1
  front = pos
  while front > 0 and index[front] is None:
    front -= 1
  assert index[front] is not None or index[rear] is not None
  if index[front] is None:
    if index[rear] >= 1:
      if is_start:
        return 0
      else:
        return index[rear] - 1
    return index[rear]
  if index[rear] is None:
    if M is not None and index[front] < M - 1:
      if is_start:
        return index[front] + 1
      else:
        return M - 1
    return index[front]
  if is_start:
    if index[rear] > index[front] + 1:
      return index[front] + 1
    else:
      return index[rear]
  else:
    if index[rear] > index[front] + 1:
      return index[rear] - 1
    else:
      return index[front]


def convert_examples_to_features(examples, sp_model, max_seq_length,
                                 doc_stride, max_query_length, is_training,
                                 output_fn):
  """Loads a data file into a list of `InputBatch`s."""

  cnt_pos, cnt_neg = 0, 0
  unique_id = 1000000000
  max_N, max_M = 1024, 1024
  f = np.zeros((max_N, max_M), dtype=np.float32)

  for (example_index, example) in enumerate(examples):

    if example_index % 100 == 0:
      tf.logging.info('Converting {}/{} pos {} neg {}'.format(
          example_index, len(examples), cnt_pos, cnt_neg))

    query_tokens = encode_ids(
        sp_model,
        preprocess_text(example.question_text, lower=FLAGS.uncased))

    if len(query_tokens) > max_query_length:
      query_tokens = query_tokens[0:max_query_length]

    paragraph_text = example.paragraph_text
    para_tokens = encode_pieces(
        sp_model,
        preprocess_text(example.paragraph_text, lower=FLAGS.uncased))

    chartok_to_tok_index = []
    tok_start_to_chartok_index = []
    tok_end_to_chartok_index = []
    char_cnt = 0
    for i, token in enumerate(para_tokens):
      chartok_to_tok_index.extend([i] * len(token))
      tok_start_to_chartok_index.append(char_cnt)
      char_cnt += len(token)
      tok_end_to_chartok_index.append(char_cnt - 1)

    tok_cat_text = ''.join(para_tokens).replace(SPIECE_UNDERLINE, ' ')
    N, M = len(paragraph_text), len(tok_cat_text)

    if N > max_N or M > max_M:
      max_N = max(N, max_N)
      max_M = max(M, max_M)
      f = np.zeros((max_N, max_M), dtype=np.float32)
      gc.collect()

    g = {}

    def _lcs_match(max_dist):
      f.fill(0)
      g.clear()

      ### longest common sub sequence
      # f[i, j] = max(f[i - 1, j], f[i, j - 1], f[i - 1, j - 1] + match(i, j))
      for i in range(N):

        # note(zhiliny):
        # unlike standard LCS, this is specifically optimized for the setting
        # because the mismatch between sentence pieces and original text will
        # be small
        for j in range(i - max_dist, i + max_dist):
          if j >= M or j < 0: continue

          if i > 0:
            g[(i, j)] = 0
            f[i, j] = f[i - 1, j]

          if j > 0 and f[i, j - 1] > f[i, j]:
            g[(i, j)] = 1
            f[i, j] = f[i, j - 1]

          f_prev = f[i - 1, j - 1] if i > 0 and j > 0 else 0
          if (preprocess_text(paragraph_text[i], lower=FLAGS.uncased,
              remove_space=False)
              == tok_cat_text[j]
              and f_prev + 1 > f[i, j]):
            g[(i, j)] = 2
            f[i, j] = f_prev + 1

    max_dist = abs(N - M) + 5
    for _ in range(2):
      _lcs_match(max_dist)
      if f[N - 1, M - 1] > 0.8 * N: break
      max_dist *= 2

    orig_to_chartok_index = [None] * N
    chartok_to_orig_index = [None] * M
    i, j = N - 1, M - 1
    while i >= 0 and j >= 0:
      if (i, j) not in g: break
      if g[(i, j)] == 2:
        orig_to_chartok_index[i] = j
        chartok_to_orig_index[j] = i
        i, j = i - 1, j - 1
      elif g[(i, j)] == 1:
        j = j - 1
      else:
        i = i - 1

    if all(v is None for v in orig_to_chartok_index) or f[N - 1, M - 1] < 0.8 * N:
      print('MISMATCH DETECTED!')
      continue

    tok_start_to_orig_index = []
    tok_end_to_orig_index = []
    for i in range(len(para_tokens)):
      start_chartok_pos = tok_start_to_chartok_index[i]
      end_chartok_pos = tok_end_to_chartok_index[i]
      start_orig_pos = _convert_index(chartok_to_orig_index, start_chartok_pos,
                                      N, is_start=True)
      end_orig_pos = _convert_index(chartok_to_orig_index, end_chartok_pos,
                                    N, is_start=False)

      tok_start_to_orig_index.append(start_orig_pos)
      tok_end_to_orig_index.append(end_orig_pos)

    if not is_training:
      tok_start_position = tok_end_position = None

    if is_training and example.is_impossible:
      tok_start_position = -1
      tok_end_position = -1

    if is_training and not example.is_impossible:
      start_position = example.start_position
      end_position = start_position + len(example.orig_answer_text) - 1

      start_chartok_pos = _convert_index(orig_to_chartok_index, start_position,
                                         is_start=True)
      tok_start_position = chartok_to_tok_index[start_chartok_pos]

      end_chartok_pos = _convert_index(orig_to_chartok_index, end_position,
                                       is_start=False)
      tok_end_position = chartok_to_tok_index[end_chartok_pos]
      assert tok_start_position <= tok_end_position

    def _piece_to_id(x):
      if six.PY2 and isinstance(x, unicode):
        x = x.encode('utf-8')
      return sp_model.PieceToId(x)

    all_doc_tokens = list(map(_piece_to_id, para_tokens))

    # The -3 accounts for [CLS], [SEP] and [SEP]
    max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
      length = len(all_doc_tokens) - start_offset
      if length > max_tokens_for_doc:
        length = max_tokens_for_doc
      doc_spans.append(_DocSpan(start=start_offset, length=length))
      if start_offset + length == len(all_doc_tokens):
        break
      start_offset += min(length, doc_stride)

    for (doc_span_index, doc_span) in enumerate(doc_spans):
      tokens = []
      token_is_max_context = {}
      segment_ids = []
      p_mask = []

      cur_tok_start_to_orig_index = []
      cur_tok_end_to_orig_index = []

      for i in range(doc_span.length):
        split_token_index = doc_span.start + i

        cur_tok_start_to_orig_index.append(
            tok_start_to_orig_index[split_token_index])
        cur_tok_end_to_orig_index.append(
            tok_end_to_orig_index[split_token_index])

        is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                               split_token_index)
        token_is_max_context[len(tokens)] = is_max_context
        tokens.append(all_doc_tokens[split_token_index])
        segment_ids.append(SEG_ID_P)
        p_mask.append(0)

      paragraph_len = len(tokens)

      tokens.append(SEP_ID)
      segment_ids.append(SEG_ID_P)
      p_mask.append(1)

      # note(zhiliny): we put P before Q
      # because during pretraining, B is always shorter than A
      for token in query_tokens:
        tokens.append(token)
        segment_ids.append(SEG_ID_Q)
        p_mask.append(1)
      tokens.append(SEP_ID)
      segment_ids.append(SEG_ID_Q)
      p_mask.append(1)

      cls_index = len(segment_ids)
      tokens.append(CLS_ID)
      segment_ids.append(SEG_ID_CLS)
      p_mask.append(0)

      input_ids = tokens

      # The mask has 0 for real tokens and 1 for padding tokens. Only real
      # tokens are attended to.
      input_mask = [0] * len(input_ids)

      # Zero-pad up to the sequence length.
      while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(1)
        segment_ids.append(SEG_ID_PAD)
        p_mask.append(1)

      assert len(input_ids) == max_seq_length
      assert len(input_mask) == max_seq_length
      assert len(segment_ids) == max_seq_length
      assert len(p_mask) == max_seq_length

      span_is_impossible = example.is_impossible
      start_position = None
      end_position = None
      if is_training and not span_is_impossible:
        # For training, if our document chunk does not contain an annotation
        # we throw it out, since there is nothing to predict.
        doc_start = doc_span.start
        doc_end = doc_span.start + doc_span.length - 1
        out_of_span = False
        if not (tok_start_position >= doc_start and
                tok_end_position <= doc_end):
          out_of_span = True
        if out_of_span:
          # continue
          start_position = 0
          end_position = 0
          span_is_impossible = True
        else:
          # note(zhiliny): we put P before Q, so doc_offset should be zero.
          # doc_offset = len(query_tokens) + 2
          doc_offset = 0
          start_position = tok_start_position - doc_start + doc_offset
          end_position = tok_end_position - doc_start + doc_offset

      if is_training and span_is_impossible:
        start_position = cls_index
        end_position = cls_index

      if example_index < 20:
        tf.logging.info("*** Example ***")
        tf.logging.info("unique_id: %s" % (unique_id))
        tf.logging.info("example_index: %s" % (example_index))
        tf.logging.info("doc_span_index: %s" % (doc_span_index))
        tf.logging.info("tok_start_to_orig_index: %s" % " ".join(
            [str(x) for x in cur_tok_start_to_orig_index]))
        tf.logging.info("tok_end_to_orig_index: %s" % " ".join(
            [str(x) for x in cur_tok_end_to_orig_index]))
        tf.logging.info("token_is_max_context: %s" % " ".join([
            "%d:%s" % (x, y) for (x, y) in six.iteritems(token_is_max_context)
        ]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info(
            "input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info(
            "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))

        if is_training and span_is_impossible:
          tf.logging.info("impossible example span")

        if is_training and not span_is_impossible:
          pieces = [sp_model.IdToPiece(token) for token in
                    tokens[start_position: (end_position + 1)]]
          answer_text = sp_model.DecodePieces(pieces)
          tf.logging.info("start_position: %d" % (start_position))
          tf.logging.info("end_position: %d" % (end_position))
          tf.logging.info(
              "answer: %s" % (printable_text(answer_text)))

          # note(zhiliny): With multi processing,
          # the example_index is actually the index within the current process
          # therefore we use example_index=None to avoid being used in the future.
          # The current code does not use example_index of training data.
      if is_training:
        feat_example_index = None
      else:
        feat_example_index = example_index

      feature = InputFeatures(
          unique_id=unique_id,
          example_index=feat_example_index,
          doc_span_index=doc_span_index,
          tok_start_to_orig_index=cur_tok_start_to_orig_index,
          tok_end_to_orig_index=cur_tok_end_to_orig_index,
          token_is_max_context=token_is_max_context,
          input_ids=input_ids,
          input_mask=input_mask,
          p_mask=p_mask,
          segment_ids=segment_ids,
          paragraph_len=paragraph_len,
          cls_index=cls_index,
          start_position=start_position,
          end_position=end_position,
          is_impossible=span_is_impossible)

      # Run callback
      output_fn(feature)

      unique_id += 1
      if span_is_impossible:
        cnt_neg += 1
      else:
        cnt_pos += 1

  tf.logging.info("Total number of instances: {} = pos {} neg {}".format(
      cnt_pos + cnt_neg, cnt_pos, cnt_neg))


def _check_is_max_context(doc_spans, cur_span_index, position):
  """Check if this is the 'max context' doc span for the token."""

  # Because of the sliding window approach taken to scoring documents, a single
  # token can appear in multiple documents. E.g.
  #  Doc: the man went to the store and bought a gallon of milk
  #  Span A: the man went to the
  #  Span B: to the store and bought
  #  Span C: and bought a gallon of
  #  ...
  #
  # Now the word 'bought' will have two scores from spans B and C. We only
  # want to consider the score with "maximum context", which we define as
  # the *minimum* of its left and right context (the *sum* of left and
  # right context will always be the same, of course).
  #
  # In the example the maximum context for 'bought' would be span C since
  # it has 1 left context and 3 right context, while span B has 4 left context
  # and 0 right context.
  best_score = None
  best_span_index = None
  for (span_index, doc_span) in enumerate(doc_spans):
    end = doc_span.start + doc_span.length - 1
    if position < doc_span.start:
      continue
    if position > end:
      continue
    num_left_context = position - doc_span.start
    num_right_context = end - position
    score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
    if best_score is None or score > best_score:
      best_score = score
      best_span_index = span_index

  return cur_span_index == best_span_index

In [0]:
def find_answer(qna_json):
  tf.logging.set_verbosity(tf.logging.INFO)
  sp_model = spm.SentencePieceProcessor()
  sp_model.Load(FLAGS.spiece_model_file)

  ### TPU Configuration
  run_config = model_utils.configure_tpu(FLAGS)

  model_fn = get_model_fn()
  spm_basename = _get_spm_basename()

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config)

  eval_examples = read_squad_examples(is_training=False,qna_json=qna_json)

  with tf.gfile.Open(FLAGS.predict_file) as f:
    orig_data = json.load(f)["data"]

  eval_rec_file = os.path.join(
      FLAGS.output_dir,
      "{}.slen-{}.qlen-{}.eval.tf_record".format(
          spm_basename, FLAGS.max_seq_length, FLAGS.max_query_length))
  eval_feature_file = os.path.join(
      FLAGS.output_dir,
      "{}.slen-{}.qlen-{}.eval.features.pkl".format(
          spm_basename, FLAGS.max_seq_length, FLAGS.max_query_length))

  if tf.gfile.Exists(eval_rec_file) and tf.gfile.Exists(
      eval_feature_file) and not FLAGS.overwrite_data:
    tf.logging.info("Loading eval features from {}".format(eval_feature_file))
    with tf.gfile.Open(eval_feature_file, 'rb') as fin:
      eval_features = pickle.load(fin)
  else:
    eval_writer = FeatureWriter(filename=eval_rec_file, is_training=False)
    eval_features = []

    def append_feature(feature):
      eval_features.append(feature)
      eval_writer.process_feature(feature)

    convert_examples_to_features(
        examples=eval_examples,
        sp_model=sp_model,
        max_seq_length=FLAGS.max_seq_length,
        doc_stride=FLAGS.doc_stride,
        max_query_length=FLAGS.max_query_length,
        is_training=False,
        output_fn=append_feature)
    eval_writer.close()

    with tf.gfile.Open(eval_feature_file, 'wb') as fout:
      pickle.dump(eval_features, fout)

  eval_input_fn = input_fn_builder(
      input_glob=eval_rec_file,
      seq_length=FLAGS.max_seq_length,
      is_training=False,
      drop_remainder=False,
      num_hosts=1)

  cur_results = []
  for result in estimator.predict(
      input_fn=eval_input_fn,
      yield_single_examples=True):

    if len(cur_results) % 1000 == 0:
      tf.logging.info("Processing example: %d" % (len(cur_results)))

    unique_id = int(result["unique_ids"])
    start_top_log_probs = (
        [float(x) for x in result["start_top_log_probs"].flat])
    start_top_index = [int(x) for x in result["start_top_index"].flat]
    end_top_log_probs = (
        [float(x) for x in result["end_top_log_probs"].flat])
    end_top_index = [int(x) for x in result["end_top_index"].flat]

    cls_logits = float(result["cls_logits"].flat[0])

    cur_results.append(
        RawResult(
            unique_id=unique_id,
            start_top_log_probs=start_top_log_probs,
            start_top_index=start_top_index,
            end_top_log_probs=end_top_log_probs,
            end_top_index=end_top_index,
            cls_logits=cls_logits))


  ret = get_predictions(eval_examples, eval_features, cur_results,
                          FLAGS.n_best_size, FLAGS.max_answer_length,
                          orig_data)

  # Log current result
  tf.logging.info("=" * 80)
  log_str = "Result | "
  for key, val in ret.items():
    log_str += "{} {} | ".format(key, val)
  tf.logging.info(log_str)
  tf.logging.info("=" * 80)


In [0]:
def get_predictions(all_examples, all_features, all_results, n_best_size,
                      max_answer_length, orig_data):
  """Write final predictions to the json file and log-odds of null if needed."""
  tf.logging.info("Generating predictions ")
  # tf.logging.info("Writing nbest to: %s" % (output_nbest_file))

  example_index_to_features = collections.defaultdict(list)
  for feature in all_features:
    example_index_to_features[feature.example_index].append(feature)

  unique_id_to_result = {}
  for result in all_results:
    unique_id_to_result[result.unique_id] = result

  all_predictions = collections.OrderedDict()
  all_nbest_json = collections.OrderedDict()
  scores_diff_json = collections.OrderedDict()

  for (example_index, example) in enumerate(all_examples):
    features = example_index_to_features[example_index]

    prelim_predictions = []
    # keep track of the minimum score of null start+end of position 0
    score_null = 1000000  # large and positive

    for (feature_index, feature) in enumerate(features):
      result = unique_id_to_result[feature.unique_id]

      cur_null_score = result.cls_logits

      # if we could have irrelevant answers, get the min score of irrelevant
      score_null = min(score_null, cur_null_score)

      for i in range(FLAGS.start_n_top):
        for j in range(FLAGS.end_n_top):
          start_log_prob = result.start_top_log_probs[i]
          start_index = result.start_top_index[i]

          j_index = i * FLAGS.end_n_top + j

          end_log_prob = result.end_top_log_probs[j_index]
          end_index = result.end_top_index[j_index]

          # We could hypothetically create invalid predictions, e.g., predict
          # that the start of the span is in the question. We throw out all
          # invalid predictions.
          if start_index >= feature.paragraph_len - 1:
            continue
          if end_index >= feature.paragraph_len - 1:
            continue

          if not feature.token_is_max_context.get(start_index, False):
            continue
          if end_index < start_index:
            continue
          length = end_index - start_index + 1
          if length > max_answer_length:
            continue

          prelim_predictions.append(
              _PrelimPrediction(
                  feature_index=feature_index,
                  start_index=start_index,
                  end_index=end_index,
                  start_log_prob=start_log_prob,
                  end_log_prob=end_log_prob))

    prelim_predictions = sorted(
        prelim_predictions,
        key=lambda x: (x.start_log_prob + x.end_log_prob),
        reverse=True)

    seen_predictions = {}
    nbest = []
    for pred in prelim_predictions:
      if len(nbest) >= n_best_size:
        break
      feature = features[pred.feature_index]

      tok_start_to_orig_index = feature.tok_start_to_orig_index
      tok_end_to_orig_index = feature.tok_end_to_orig_index
      start_orig_pos = tok_start_to_orig_index[pred.start_index]
      end_orig_pos = tok_end_to_orig_index[pred.end_index]

      paragraph_text = example.paragraph_text
      final_text = paragraph_text[start_orig_pos: end_orig_pos + 1].strip()

      if final_text in seen_predictions:
        continue

      seen_predictions[final_text] = True

      nbest.append(
          _NbestPrediction(
              text=final_text,
              start_log_prob=pred.start_log_prob,
              end_log_prob=pred.end_log_prob))

    # In very rare edge cases we could have no valid predictions. So we
    # just create a nonce prediction in this case to avoid failure.
    if not nbest:
      nbest.append(
          _NbestPrediction(text="", start_log_prob=-1e6,
          end_log_prob=-1e6))

    total_scores = []
    best_non_null_entry = None
    for entry in nbest:
      total_scores.append(entry.start_log_prob + entry.end_log_prob)
      if not best_non_null_entry:
        best_non_null_entry = entry

    probs = _compute_softmax(total_scores)

    nbest_json = []
    for (i, entry) in enumerate(nbest):
      output = collections.OrderedDict()
      output["text"] = entry.text
      output["probability"] = probs[i]
      output["start_log_prob"] = entry.start_log_prob
      output["end_log_prob"] = entry.end_log_prob
      nbest_json.append(output)

    assert len(nbest_json) >= 1
    assert best_non_null_entry is not None

    score_diff = score_null
    scores_diff_json[example.qas_id] = score_diff
    # note(zhiliny): always predict best_non_null_entry
    # and the evaluation script will search for the best threshold
    all_predictions[example.qas_id] = best_non_null_entry.text

    all_nbest_json[example.qas_id] = nbest_json
    all_predictions_json = json.dumps(all_predictions, indent=4)

    return all_predictions_json, all_nbest_json


def _get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes


def _compute_softmax(scores):
  """Compute softmax probability over raw logits."""
  if not scores:
    return []

  max_score = None
  for score in scores:
    if max_score is None or score > max_score:
      max_score = score

  exp_scores = []
  total_sum = 0.0
  for score in scores:
    x = math.exp(score - max_score)
    exp_scores.append(x)
    total_sum += x

  probs = []
  for score in exp_scores:
    probs.append(score / total_sum)
  return probs


def input_fn_builder(input_glob, seq_length, is_training, drop_remainder,
                     num_hosts, num_threads=8):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "unique_ids": tf.FixedLenFeature([], tf.int64),
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.float32),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "cls_index": tf.FixedLenFeature([], tf.int64),
      "p_mask": tf.FixedLenFeature([seq_length], tf.float32)
  }

  if is_training:
    name_to_features["start_positions"] = tf.FixedLenFeature([], tf.int64)
    name_to_features["end_positions"] = tf.FixedLenFeature([], tf.int64)
    name_to_features["is_impossible"] = tf.FixedLenFeature([], tf.float32)

  tf.logging.info("Input tfrecord file glob {}".format(input_glob))
  global_input_paths = tf.gfile.Glob(input_glob)
  tf.logging.info("Find {} input paths {}".format(
      len(global_input_paths), global_input_paths))

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.cast(t, tf.int32)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    if FLAGS.use_tpu:
      batch_size = params["batch_size"]
    elif is_training:
      batch_size = FLAGS.train_batch_size
    else:
      batch_size = FLAGS.predict_batch_size

    # Split tfrecords across hosts
    if num_hosts > 1:
      host_id = params["context"].current_host
      num_files = len(global_input_paths)
      if num_files >= num_hosts:
        num_files_per_host = (num_files + num_hosts - 1) // num_hosts
        my_start_file_id = host_id * num_files_per_host
        my_end_file_id = min((host_id + 1) * num_files_per_host, num_files)
        input_paths = global_input_paths[my_start_file_id: my_end_file_id]
      tf.logging.info("Host {} handles {} files".format(host_id,
                                                        len(input_paths)))
    else:
      input_paths = global_input_paths

    if len(input_paths) == 1:
      d = tf.data.TFRecordDataset(input_paths[0])
      # For training, we want a lot of parallel reading and shuffling.
      # For eval, we want no shuffling and parallel reading doesn't matter.
      if is_training:
        d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)
        d = d.repeat()
    else:
      d = tf.data.Dataset.from_tensor_slices(input_paths)
      # file level shuffle
      d = d.shuffle(len(input_paths)).repeat()

      # `cycle_length` is the number of parallel files that get read.
      cycle_length = min(num_threads, len(input_paths))

      d = d.apply(
          tf.contrib.data.parallel_interleave(
              tf.data.TFRecordDataset,
              sloppy=is_training,
              cycle_length=cycle_length))

      if is_training:
        # sample level shuffle
        d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            num_parallel_batches=num_threads,
            drop_remainder=drop_remainder))
    d = d.prefetch(1024)

    return d

  return input_fn


def get_model_fn():
  def model_fn(features, labels, mode, params):
    #### Training or Evaluation
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    #### Get loss from inputs
    outputs = function_builder.get_qa_outputs(FLAGS, features, is_training)

    #### Check model parameters
    num_params = sum([np.prod(v.shape) for v in tf.trainable_variables()])
    tf.logging.info('#params: {}'.format(num_params))

    scaffold_fn = None

    #### Evaluation mode
    if mode == tf.estimator.ModeKeys.PREDICT:
      if FLAGS.init_checkpoint:
        tf.logging.info("init_checkpoint not being used in predict mode.")

      predictions = {
          "unique_ids": features["unique_ids"],
          "start_top_index": outputs["start_top_index"],
          "start_top_log_probs": outputs["start_top_log_probs"],
          "end_top_index": outputs["end_top_index"],
          "end_top_log_probs": outputs["end_top_log_probs"],
          "cls_logits": outputs["cls_logits"]
      }

      if FLAGS.use_tpu:
        output_spec = tf.contrib.tpu.TPUEstimatorSpec(
            mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
      else:
        output_spec = tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions)
      return output_spec

    ### Compute loss
    seq_length = tf.shape(features["input_ids"])[1]
    def compute_loss(log_probs, positions):
      one_hot_positions = tf.one_hot(
          positions, depth=seq_length, dtype=tf.float32)

      loss = - tf.reduce_sum(one_hot_positions * log_probs, axis=-1)
      loss = tf.reduce_mean(loss)
      return loss

    start_loss = compute_loss(
        outputs["start_log_probs"], features["start_positions"])
    end_loss = compute_loss(
        outputs["end_log_probs"], features["end_positions"])

    total_loss = (start_loss + end_loss) * 0.5

    cls_logits = outputs["cls_logits"]
    is_impossible = tf.reshape(features["is_impossible"], [-1])
    regression_loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=is_impossible, logits=cls_logits)
    regression_loss = tf.reduce_mean(regression_loss)

    # note(zhiliny): by default multiply the loss by 0.5 so that the scale is
    # comparable to start_loss and end_loss
    total_loss += regression_loss * 0.5

    #### Configuring the optimizer
    train_op, learning_rate, _ = model_utils.get_train_op(FLAGS, total_loss)

    monitor_dict = {}
    monitor_dict["lr"] = learning_rate

    #### load pretrained models
    scaffold_fn = model_utils.init_from_checkpoint(FLAGS)

    #### Constucting training TPUEstimatorSpec with new cache.
    if FLAGS.use_tpu:
      host_call = function_builder.construct_scalar_host_call(
          monitor_dict=monitor_dict,
          model_dir=FLAGS.model_dir,
          prefix="train/",
          reduce_fn=tf.reduce_mean)

      train_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, loss=total_loss, train_op=train_op, host_call=host_call,
          scaffold_fn=scaffold_fn)
    else:
      train_spec = tf.estimator.EstimatorSpec(
          mode=mode, loss=total_loss, train_op=train_op)

    return train_spec

  return model_fn


def _get_spm_basename():
  spm_basename = os.path.basename(FLAGS.spiece_model_file)
  return spm_basename


def preprocess():
  sp_model = spm.SentencePieceProcessor()
  sp_model.Load(FLAGS.spiece_model_file)
  spm_basename = _get_spm_basename()

  train_rec_file = os.path.join(
      FLAGS.output_dir,
      "{}.{}.slen-{}.qlen-{}.train.tf_record".format(
          spm_basename, FLAGS.proc_id, FLAGS.max_seq_length,
          FLAGS.max_query_length))

  tf.logging.info("Read examples from {}".format(FLAGS.train_file))
  train_examples = read_squad_examples(FLAGS.train_file, is_training=True)
  train_examples = train_examples[FLAGS.proc_id::FLAGS.num_proc]

  # Pre-shuffle the input to avoid having to make a very large shuffle
  # buffer in the `input_fn`.
  random.shuffle(train_examples)

  tf.logging.info("Write to {}".format(train_rec_file))
  train_writer = FeatureWriter(
      filename=train_rec_file,
      is_training=True)
  convert_examples_to_features(
      examples=train_examples,
      sp_model=sp_model,
      max_seq_length=FLAGS.max_seq_length,
      doc_stride=FLAGS.doc_stride,
      max_query_length=FLAGS.max_query_length,
      is_training=True,
      output_fn=train_writer.process_feature)
  train_writer.close()

In [29]:
result = get_answer(
        '''Avengers: Endgame is a 2019 American superhero film based on the Marvel Comics superhero team the Avengers, produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. It is the sequel to 2012\'s The Avengers, 2015\'s Avengers: Age of Ultron, and 2018\'s Avengers: Infinity War and the twenty-second film in the Marvel Cinematic Universe (MCU). It was directed by Anthony and Joe Russo, written by Christopher Markus and Stephen McFeely, and features an ensemble cast including Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner, Don Cheadle, Paul Rudd, Brie Larson, Karen Gillan, Danai Gurira, Bradley Cooper, and Josh Brolin. In the film, the surviving members of the Avengers and their allies attempt to reverse the damage caused by Thanos in Infinity War.\nThe film was announced in October 2014 as Avengers: Infinity War – Part 2. The Russo brothers came on board to direct in April 2015, and by May, Markus and McFeely signed on to write the script for the film. In July 2016, Marvel removed the film\'s title, and it remained untitled until its official title was revealed in December 2018. Filming began in August 2017 at Pinewood Atlanta Studios in Fayette County, Georgia, shooting back-to-back with Infinity War, and ended in January 2018. Additional filming took place in the Metro and Downtown Atlanta areas, New York, and Scotland. With an estimated budget of $356 million, it is the third-most expensive film ever made.\nAvengers: Endgame was widely anticipated, and Disney backed the film with extensive marketing campaigns. It premiered in Los Angeles on April 22, 2019, and was theatrically released in the United States on April 26, 2019, in IMAX and 3D. The film received praise for its direction, acting, musical score, visual effects, and emotional weight, with critics lauding its culmination of the 22-film story. It has grossed over $2.7 billion worldwide, surpassing Infinity War\'s entire theatrical run in just eleven days and setting numerous highest-grossing film and box office records, and is the highest-grossing film of 2019, as well as the second-highest of all time.\n\n\n== Plot ==\nTwenty-three days after Thanos used the Infinity Gauntlet to disintegrate half of all life in the universe, Carol Danvers rescues Tony Stark and Nebula from deep space and returns them to Earth. They reunite with the remaining Avengers—Bruce Banner, Steve Rogers, Thor, Natasha Romanoff, and James Rhodes—and Rocket, and they find Thanos on an otherwise uninhabited planet. They plan to retake and use the Infinity Stones to reverse the disintegrations, but Thanos reveals he destroyed them to prevent further use. An enraged Thor decapitates Thanos.\nFive years later, Scott Lang escapes from the quantum realm. At the Avengers compound, he explains to Romanoff and Rogers that he experienced only five hours while trapped. Theorizing the quantum realm could allow time travel, the three ask Stark to help them retrieve the Stones from the past to reverse Thanos\' actions in the present. Stark refuses, thinking of his wife, Pepper Potts, and daughter, Morgan, but relents after looking at a picture of himself and Peter Parker. Stark and Banner, who has since merged his intelligence with the Hulk\'s strength, build a time machine. Banner notes that changing the past does not affect their present and any changes instead create branched alternate realities. He and Rocket visit the Asgardian refugees\' new home in Norway—New Asgard—to recruit Thor, now an overweight alcoholic, despondent over his failure to stop Thanos. In Tokyo, Romanoff recruits Clint Barton, now a vigilante following the disintegration of his family.\nBanner, Lang, Rogers, and Stark travel to New York City in 2012. Banner visits the Sanctum Sanctorum and convinces the Ancient One to give him the Time Stone. At Stark Tower, Rogers retrieves the Mind Stone, but Stark and Lang\'s attempt to steal the Space Stone fails, allowing 2012 Loki to escape with it. Rogers and Stark travel to S.H.I.E.L.D. headquarters in 1970, where Stark obtains an earlier version of the Space Stone and encounters his father, Howard, while Rogers steals several Pym Particles from Hank Pym to return to the present. Meanwhile, Rocket and Thor travel to Asgard in 2013, extracting the Reality Stone from Jane Foster and retrieving Thor\'s hammer, Mjolnir. Nebula and Rhodes travel to Morag in 2014 and steal the Power Stone before Peter Quill can. Rhodes returns to the present with the Power Stone, but Nebula is incapacitated when her cybernetic implants link with those of her past self. Through this connection, 2014 Thanos learns of his future success and the Avengers\' attempts to undo it. He captures Nebula and sends 2014 Nebula to the present in the former\'s place. Barton and Romanoff travel to Vormir, where the Soul Stone\'s keeper, the Red Skull, reveals it can only be acquired by sacrificing someone they love. Romanoff sacrifices herself, allowing Barton to obtain the Soul Stone.\nReuniting in the present, the Avengers fit the Stones into a Stark-created gauntlet, which Banner uses to resurrect the disintegrated. 2014 Nebula uses the time machine to transport 2014 Thanos and his warship to the present, where he attacks the Avengers\' compound, planning to destroy and rebuild the universe with the Stones. Nebula convinces 2014 Gamora to betray Thanos, but fails to convince 2014 Nebula and is forced to kill her. Stark, Rogers, and Thor fight Thanos but are outmatched. Thanos summons his army to devastate Earth, but a restored Stephen Strange arrives with other sorcerers, the restored Avengers and Guardians of the Galaxy, the armies of Wakanda and Asgard, and the Ravagers to fight Thanos and his army alongside Danvers, who destroys Thanos\' warship as she arrives. After overpowering the heroes, Thanos seizes the gauntlet, but Stark steals the Stones back and uses them to disintegrate Thanos and his army, at the cost of his life.\nFollowing Stark\'s funeral, Thor appoints Valkyrie as the new ruler of New Asgard and joins the Guardians of the Galaxy. Rogers returns the Infinity Stones and Mjolnir to their original timelines and remains in the past to live with Peggy Carter. In the present, an elderly Rogers passes on his shield and mantle to Sam Wilson.\n\n\n== Cast ==\nRobert Downey Jr. as Tony Stark / Iron Man: The benefactor and a member of the Avengers who aids them by using electromechanical suits of armor of his own making. According to directors Joe and Anthony Russo, Downey was one of the few actors to read the entire screenplay for the film. Screenwriters Christopher Markus and Stephen McFeely knew that the film would inevitably feature Stark\'s death both as a "[move] to selflessness" and as an end to the "chapter" Stark started. They felt that his death was earned after granting him "the perfect retirement life," adding, "That’s the life he’s been striving for...They got married, they had a kid, it was great. It’s a good death. It doesn’t feel like a tragedy. It feels like a heroic, finished life." Joe Russo explained that Stark "always knew he was going to die because he could never reconcile that notion in himself of not protecting the universe," and added that Stark was the most defiant among the Avengers since "Stark is the most formidable of all of them...because of his heart.” The Russos sought Downey\'s approval for Stark\'s arc, which they had developed since Captain America: Civil War (2016).\nChris Evans as Steve Rogers / Captain America: The leader of the Avengers. A World War II veteran, he was enhanced to the peak of human physicality by an experimental serum and frozen in suspended animation before waking up in the modern world. Markus described Rogers as someone who is "moving toward some sort of enlightened self-interest." Both he and McFeely knew he was going to get "the dance" he promised Peggy Carter in Captain America: The First Avenger (2011), with McFeely saying, "He’s postponed a life in order to fulfill his duty. That’s why I didn’t think we were ever going to kill him. Because that’s not the arc. The arc is, I finally get to put my shield down because I’ve earned that."  Patrick Gorman plays an elderly Steve Rogers.\nMark Ruffalo as Bruce Banner / Hulk: An Avenger and genius scientist who, because of exposure to gamma radiation, possesses enhanced strength and a monstrous appearance. In this film, Banner has managed to balance his two sides with gamma experimentation, enabling him to combine his intelligence with the Hulk\'s strength and physical stature, based on the "Professor Hulk" comic book identity. Compared to other heroes, who were demotivated by their loss against Thanos, Banner is the only character to remain hopeful, with Anthony Russo explaining, "Banner is the sole character who is actually forging into a bright new future, trying to build something totally new and find something completely new...Banner is the one who is most heroic in a sense that he maintains his will to keep trying.” This concludes a character arc that was established in Thor: Ragnarok and continued in Avengers: Infinity War.\nChris Hemsworth as Thor: An Avenger and the king of New Asgard, based on the Norse mythological deity of the same name. Thor now wields a mystical axe known as Stormbreaker, after the destruction of his hammer, Mjolnir, in Thor: Ragnarok. In the film, Thor has become an overweight, drunken ruler of Asgard\'s refugees in Tønsberg, Norway. Referencing this drastic character change, Hemsworth said, "I just had an opinion. I wanted to do something different this time. Each film I’ve wanted to, in particular, the last couple, and they were on board," and added, "We shot for many hours and days and discussed how far could we push (Thor) and what we could do different." Anthony Russo added, "Even though there\'s a lot of fun to be had in the movie with his physical condition, it\'s not a gag. It\'s a manifestation of where he is on a character level, and we think it\'s one of the most relatable aspects of him. I mean, it\'s a very common sort of response to depression and pain." Thor\'s story was his favorite arc, saying, “Part of Chris’ magic as a comedic actor is his dedication to the depth of the character on a very earnest level...It’s so devious and subversive when comedy is coming from a place of complete commitment and emotional complexity.” Hemsworth underwent around three hours of hair and makeup for the transformation, which also required him to wear a large silicone prosthetic suit; he called himself "Lebowski Thor" on set. Thor was initially supposed to revert back to his "old chiseled self" in the middle of Endgame, but Hemsworth successfully argued to keep Thor\'s new body.\nScarlett Johansson as Natasha Romanoff / Black Widow: A highly trained spy, Avenger, and former agent of S.H.I.E.L.D. At the beginning of the film, Romanoff continues to command several teams from around the galaxy in the Avengers headquarters, which Joe Russo explained was stemmed from her inability to move on from their failure to stop Thanos, saying, "she’s doing everything she can to try and hold the community together...She’s the watcher on the wall still." On the decision for Romanoff to sacrifice herself for Barton to acquire the Soul Stone to bring back everyone, Joe Russo stated that it was part of a larger theme exploring the desire to sacrifice, compared to the desire to protect in Infinity War; he says, "When she gets to that [Soul Stone] scene, I think she understands that the only way to bring the community back is for her to sacrifice herself." McFeely stated, "Her journey, in our minds, had come to an end if she could get the Avengers back. She comes from such an abusive, terrible, mind-control background, so when she gets to Vormir and she has a chance to get the family back, that’s a thing she would trade for." To prepare for the film, Johansson underwent a high-intensity workout regimen, which included plyometrics, Olympic weightlifting and gymnastics, as well as a time-restricted eating diet; all are under the supervision of her longtime trainer, Eric Johnson, with whom she had worked since Iron Man 2 (2010), the film which introduced her character.\nJeremy Renner as Clint Barton / Hawkeye: A master archer, Avenger, and former agent of S.H.I.E.L.D. who has become a vigilante known as Ronin, following the disappearance of his family. McFeely described Barton\'s dark turn as "a good example of people who had much stronger stories after the Snap." The film\'s cold open, which features the disintegration of Barton\'s family, was initially supposed to be in Infinity War following Thanos\' snap, however it was moved to Endgame instead, with Markus explaining that it was "going to blunt the brutality of what [Thanos] did." Joe Russo felt it was "a very tragic scene to open the movie with. It\'s one of the few scenes in the movie that actually makes me tear up when I watch it, because I think about my own family...And then you think about what would happen to you, as a father. You\'d become very self-destructive.”\nDon Cheadle as James "Rhodey" Rhodes / War Machine: A former officer in the U.S. Air Force and Avenger who operates the War Machine armor. Cheadle described Rhodes\' newfound belonging as an Avenger as "not so much straddling one foot in the military. He’s much more on the side of The Avengers than he was prior." This is reflected on Rhodes\' more instinctive and realist worldview in the midst of encountering the fantastic, with Cheadle explaining, "He’s definitely got some \'what-the-eff-is-happening\' [attitude,] more than maybe the rest of them do, given his background. But it’s a trial by fire, and he’s quickly adapted to what [the threat] is, rather than what he wishes it were."\nPaul Rudd as Scott Lang / Ant-Man: An Avenger and former petty criminal who acquired a suit that allows him to shrink or grow in scale and increase in strength. Lang is portrayed by twins Bazlo and Loen LeClair as a baby, by Jackson A. Dunn at age 12, and by Lee Moore at age 89. This was Moore\'s final film before his death in August 2018. Markus explained that adding Lang helped with implementing time travel into the film, saying, "we had access to him in the second movie, and the fact that he was bringing a whole subset of technology that did have something to do with a different concept of time was like a birthday present."\nBrie Larson as Carol Danvers / Captain Marvel: An ex-U.S. Air Force pilot and new Avenger whose DNA was altered during an accident, imbuing her with superhuman strength, energy projection, and flight. Markus stated that Danvers\' powers are on a scale that has not previously existed in the MCU and likened her personality to Rogers\', "which is sort of a person who\'s right and knows they\'re right and doesn\'t really want to hear it when you tell them they\'re wrong". Danvers has little screen time in the film, which McFeely reasoned as "not the story we’re trying to tell — it’s the original Avengers dealing with loss and coming to a conclusion, and she’s the new, fresh blood." Larson filmed her scenes for Endgame before beginning work on her solo film Captain Marvel, which was released first. Captain Marvel directors Anna Boden and Ryan Fleck were present for the filming of her scenes in Endgame and gave Danvers\' characterization in the film their blessing.\nKaren Gillan as Nebula: A member of the Guardians of the Galaxy, Avenger, and adopted daughter of Thanos who was raised with Gamora as siblings. After being previously featured as an antagonist or an anti-hero in previous MCU films, Nebula undergoes a redemption arc in the film where she makes amends for her past actions, including an encounter with a past version of herself, with Gillan adding, "[She’s] staring her former self in the face and it’s really clear how far she’s come from that angry, bitter and twisted person. She’s starting to connect with other people and find some level of forgiveness." Gillan guessed that Nebula would play a prominent role in the film when she realized that Infinity War and Endgame would be adapted from The Infinity Gauntlet, which she had previously read when she was initially cast as Nebula in Guardians of the Galaxy (2014). Gillan shared several scenes with Downey in the film\'s opening, and the two improvised most of their scenes together.\nDanai Gurira as Okoye: A Wakandan traditionalist from the Border Tribe who is the head of the Dora Milaje, Wakanda\'s all-female special forces and T\'Challa\'s bodyguard.\nBradley Cooper as Rocket: A member of the Guardians of the Galaxy and now Avenger who is a genetically-engineered raccoon-based bounty hunter, mercenary, and master of weapons and battle tactics. Sean Gunn was again the stand-in for Rocket during filming, with his acting and expressions serving as motion reference for the character. Rocket\'s appearance in the film continues a story arc that was established by Guardians of the Galaxy writer-director and Endgame executive producer James Gunn in the first two Guardians of the Galaxy films, was continued in Infinity War and Endgame, and will conclude in the upcoming Guardians of the Galaxy Vol. 3.\nGwyneth Paltrow as Virginia "Pepper" Potts: Stark\'s wife and the CEO of Stark Industries who wears a weaponized powered exosuit of armor made for her by Stark, based on the Rescue armor. Paltrow said this would be her final major appearance in the MCU.\nJosh Brolin as Thanos: An intergalactic warlord from Titan who collected the six Infinity Stones to impose his will on all life in the universe. Joe Russo said that after Thanos was successful in Avengers: Infinity War, he is now "done. He did it. He\'s retired." Markus and McFeely had difficulty in factoring the older, post-Infinity War, Thanos into the film due to the character already possessing the Infinity Stones, until executive producer Trinh Tran suggested that they kill Thanos in the film\'s first act. Markus explained that the character\'s early death "reinforced Thanos’s agenda. He was done...it was like, \'If I’ve got to die, I can die now.\'" Thanos has less screen time in Endgame than in Infinity War, where he was considered the main character, as explained by McFeely: “We had to give ourselves permission to backseat the villain a little bit. I don\'t think anyone in the first half of the movie is going, \'Oh I wish there was a villain\'. You\'re rolling around in the loss and the time heist, and you think it\'s sort of Avengers against nature." The younger version of Thanos was nicknamed "Warrior Thanos" by the filmmakers. In addition to providing the voice for the character, Brolin performed motion capture on set.Several actors reprise their roles in Infinity War, including Benedict Cumberbatch as Dr. Stephen Strange, Chadwick Boseman as T\'Challa / Black Panther, Tom Holland as Peter Parker / Spider-Man, Zoe Saldana as Gamora, Evangeline Lilly as Hope van Dyne / Wasp, Elizabeth Olsen as Wanda Maximoff / Scarlet Witch, Anthony Mackie as Sam Wilson / Falcon, Sebastian Stan as Bucky Barnes / Winter Soldier, Tom Hiddleston as Loki, Pom Klementieff as Mantis, Dave Bautista as Drax the Destroyer, Letitia Wright as Shuri, Cobie Smulders as Maria Hill, Tom Vaughan-Lawlor as Ebony Maw, Vin Diesel as Groot, Chris Pratt as Peter Quill / Star-Lord, Samuel L. Jackson as Nick Fury, Michael James Shaw as Corvus Glaive, and Terry Notary as Cull Obsidian. Monique Ganderton provides the motion capture performance for Proxima Midnight, having previously played the role with Carrie Coon.Also reprising their roles from previous MCU films, including characters who died before the events of Infinity War, are Tessa Thompson as Valkyrie, Rene Russo as Frigga, John Slattery as Howard Stark, Tilda Swinton as the Ancient One, Benedict Wong as Wong, Jon Favreau as Harold "Happy" Hogan, Hayley Atwell as Peggy Carter, Marisa Tomei as May Parker, Taika Waititi as Korg, Angela Bassett as Ramonda, Michael Douglas as Hank Pym, Michelle Pfeiffer as Janet van Dyne, William Hurt as Thaddeus Ross, Winston Duke as M\'Baku, Linda Cardellini as Laura Barton, Maximiliano Hernández as Jasper Sitwell, Frank Grillo as Brock Rumlow, Jacob Batalon as Ned, Robert Redford as Alexander Pierce, Ross Marquand as Red Skull/Stonekeeper, Kerry Condon as the voice of Stark\'s suit A.I. F.R.I.D.A.Y, Callan Mulvey as Jack Rollins, and Ty Simpkins as Harley Keener. Sean Gunn reprised his role as Kraglin Obfonteri, though it is uncertain if footage of his appearance was used. Natalie Portman appears as Jane Foster through the use of a deleted scene from Thor: The Dark World and new voice over that Portman recorded for when Foster appears talking in the distance. James D\'Arcy reprises his role as Edwin Jarvis from the MCU television series Agent Carter, marking the first time a character introduced in an MCU television series appears in an MCU film.Additionally, Hiroyuki Sanada portrays Akihiko, a Yakuza boss operating in Tokyo who opposes Barton. Alexandra Rabe portrays Morgan Stark, Tony and Pepper\'s daughter. Katherine Langford was cast to portray an older Morgan, but her scene was deleted from the film. Emma Fuhrmann portrays present-day Cassie Lang, Scott\'s daughter. Avengers co-creator Stan Lee has a posthumous cameo in the film, appearing digitally de-aged as a car driver in 1970; this is his final film appearance. Ken Jeong and Yvette Nicole Brown cameo as a storage facility guard and a S.H.I.E.L.D. agent, respectively. Co-director Joe Russo (credited as Gozie Agbo) has a cameo appearance as a gay man grieving the sudden loss of a loved one, which is the first time an openly homosexual character has been portrayed in an MCU film. Joe\'s daughters Ava and Lia respectively portray Barton\'s daughter Lila and a fan of Hulk. Thanos creator Jim Starlin also appears as a grieving man. The character Howard the Duck appears in a non-speaking cameo.\n\n\n== Production ==\n\nIn October 2014, Marvel announced a two-part sequel to Age of Ultron, titled Avengers: Infinity War. Part 1 was scheduled to be released on May 4, 2018, with Part 2 scheduled for May 3, 2019. In April 2015, Marvel announced that Anthony and Joe Russo would direct both parts of Avengers: Infinity War, with back-to-back filming expected to begin in 2016. Also in the month, Kevin Feige said the films were titled as two parts of a single film "because they [have] such shared elements", but he would not actually describe them as "one story that\'s cut in half. I would say it\'s going to be two distinct movies." By May 2015, Christopher Markus and Stephen McFeely signed on to write the screenplays for both parts of the film. The following May, the Russos revealed that they would be retitling the two films, to further remove the misconception that the films were one large film split in two, with Joe stating, "The intention is we will change [the titles], we just haven\'t come up with [them] yet." In July 2016, Marvel removed the film\'s title, simply referring to it as Untitled Avengers film. Feige and the Russo brothers indicated the title was being withheld because it would give away plot details for this film and Infinity War.Principal photography began on August 10, 2017, under the working title Mary Lou 2, at Pinewood Atlanta Studios in Fayette County, Georgia, with Trent Opaloch serving as director of photography. The film, along with Infinity War, were shot using IMAX/Arri 2D cameras, thus marking the first time that a Hollywood feature film was shot entirely with IMAX digital cameras. Also in the month, filming occurred in The Gulch area of Downtown Atlanta, near the Five Points MARTA station, and Piedmont Park. Feige explained that the films were originally scheduled to be filmed simultaneously but were ultimately shot back-to-back, as "It became too complicated to cross-board them like that, and we found ourselves—again, something would always pay the price. We wanted to be able to focus and shoot one movie and then focus and shoot another movie." Anthony Russo originally felt it made more sense to shoot the films simultaneously due to financial and logistical reasons considering the large number of cast members, even though each part is its own distinct film, and suggested that "some days we\'ll be shooting the first movie and some days we\'ll be shooting the second movie. Just jumping back and forth." Production wrapped on January 11, 2018, although additional filming took place in Dutchess and Ulster counties in New York in June 2018. Reshoots began by September 7, 2018, and concluded on October 12, 2018. More reshoots occurred in January 2019. Location shooting also took place in St Abbs, Scotland, which doubled for New Asgard in Norway. Evans and Hemsworth each earned $15 million for the film.On December 7, 2018, with the release of the film\'s first trailer, the title was revealed to be Avengers: Endgame, while also moving its release date in the United States to April 26, 2019. Visual effects for the film were created by Industrial Light & Magic, Weta Digital, DNEG, Framestore, Cinesite, Digital Domain, Rise, Lola VFX, Cantina Creative, Capital T, Technicolor VFX, and Territory Studio. As with previous MCU films, Lola once again worked on the de-aging sequences. The 1970 New Jersey sequences featuring a young Hank Pym referenced Michael Douglas\' appearance in The Streets of San Francisco. Jeffrey Ford and Matthew Schmidt served as the film\'s editors.\n\n\n== Music ==\n\nIn June 2016, Alan Silvestri, who composed the score for The Avengers, was revealed to be returning to score both Infinity War and Endgame. The Russos started working with Silvestri on the Endgame score in early November 2018, and it was completed in late March 2019. Silvestri described the score as having the most versatile tone of the franchise, ranging from "thunderous percussion and powerful brass" for the action sequences to minimalist, jazz-inspired music for Ant-Man and the quantum realm. Silvestri reprises his themes from the previous Avengers films and Captain America: The First Avenger, such as material he wrote for Thanos and the Infinity Stones in Infinity War. He found writing the music to end Captain America\'s story poignant, since he had "been on this journey with him since the beginning". The film\'s score also uses the Ant-Man theme by Christophe Beck, the Doctor Strange theme by Michael Giacchino and the Captain Marvel theme by Pinar Toprak. Additionally, the songs "Come and Get Your Love" by Redbone and "It\'s Been a Long, Long Time" by Jule Styne and Sammy Cahn are used, after previously being heard in Guardians of the Galaxy and Captain America: The Winter Soldier, respectively. A soundtrack album featuring Silvestri\'s score was released by Hollywood Records digitally on April 26, 2019, with a physical release on May 24.A music video for the track "Portals", composed for the scene in which Doctor Strange and Wong gather reinforcements for the Avengers in the final showdown against Thanos, was released on June 13.\n\n\n== Marketing ==\nThe marketing campaign for Endgame is the largest for any Marvel Studios film, surpassing $200 million and beating Infinity War\'s $150 million campaign. Promotional partners include Stand Up to Cancer, Mastercard, Ulta Beauty, the Audi e-tron GT Concept car (which is also featured in the film), McDonald\'s, GEICO, Coca-Cola, Google, General Mills, Hertz, Ziploc, Oppo, and Synchrony Financial.A year prior to the film\'s release, Germain Lussier of io9 spoke on the approach Marvel might have to take in marketing the film, given the end of Infinity War, which sees many established characters die. He questioned if those characters would appear on posters and in toy campaigns and if the actors portraying them would participate in press events leading up to the film\'s release. Lussier felt Disney and Marvel could focus on the original Avengers team members, who make up the majority of the living characters, but noted it would be more beneficial to show the return of the dead characters, which would "build a mystery and curiosity about how they come back. It could create a whole new level of interest about the film while having all the stars front and center, as they should be".In June 2018, Feige spoke to this, stating that these dead characters would not be featured in any marketing for the film, though the decision regarding that could change. He presented a behind-the-scenes video from the film at CineEurope, and said that the official marketing campaign for the film would begin at the end of 2018 with the announcement of the film\'s title then. In early December 2018, ahead of the first trailer\'s release, Graeme McMillan of The Hollywood Reporter spoke to the "fevered anticipation" surrounding it. McMillan felt what was so remarkable about the anticipation, was that it was mostly "fan-created, without noticeable direction from Marvel or the filmmakers involved" and that the amount of knowledge surrounding the film without any type of promotion was "a kind of brand awareness that most studios would kill for". Because of this, McMillan urged Marvel not to release any trailers for the film since "the advanced level of enthusiasm that\'s already out there for the movie... is only likely to build as it gets closer to" the film\'s release. That said, he added that the eventual release of the trailer would take away the "Schrödinger\'s cat-esque position... it currently enjoys" as it was "almost guaranteed to disappoint fans, who have by this point built up their own personal trailers filled with whatever moments are essential to their enjoyment of a good teaser".The first trailer for the film was released on December 7, 2018. Dustin Sandoval, vice president of digital marketing for Marvel Studios, stated the marketing team "actively made the choice not to include the title or hashtag of the movie in our trailer posts so fans could see it at the end reveal without it being spoiled". Richard Newby, also of The Hollywood Reporter felt, while not much new was revealed in the trailer, it "offers a somber glimpse of a universe made unrecognizable" and lets the viewer "sit, rightfully so, with the ending of Avengers: Infinity War and our questions of loss". Newby also noted how the trailer "calls back to beginnings of the MCU... [as] the visual language used in the trailer does a great service by highlighting these characters\' humble beginnings", and concluded it leaves viewers with "just as many questions as we had before". Austen Goslin of Polygon pointed out that the title not only references a line Doctor Strange tells Tony Stark in Infinity War, but also a line spoken by Stark in Avengers: Age of Ultron. Goslin, who felt this was "probably [not] an accident", said, "The scene surrounding this line in Age of Ultron is one of the most important ones in the movie. Things look dark, and the group of heroes face an enemy they don\'t think they can defeat." As such, the Endgame trailer "mirrors this perfectly" and "shows us that the Avengers\' two most prominent characters are who they\'ve always been: Iron Man, a pessimist who keeps fighting no matter how hopeless things look, and Captain America, an optimist who believes that nothing is hopeless when the world\'s heroes fight together". The trailer was viewed 289 million times in its first 24 hours, becoming the most viewed trailer in that time period, surpassing the record of Avengers: Infinity War (230 million views). The trailer also set a record for Twitter conversation for a film trailer in the first 24 hours generating 549,000 mentions. By January 3, 2019, BoxOffice revealed their "Trailer Impact" metric service indicated approximately 77–78% of people surveyed who viewed the Endgame trailer in the past three weeks expressed interest in seeing the film. In the three weeks it was measured by "Trailer Impact", it was number one for all, and had the top two percent of respondents express interest in seeing the film since the service\'s introduction in March 2018.The second trailer for the film, along with the theatrical release poster, was released on March 14, 2019. The poster, featuring 13 characters, had 12 of those actors listed on the top-billing, with Danai Gurira excluded. Despite Gurira\'s name appearing in the poster\'s bottom billing block along with Benedict Wong, Jon Favreau, and Gwyneth Paltrow (none of whom were featured on the poster), her exclusion in the top billing prompted criticism from fans. Petrana Radulovic of Polygon noted how an actor is credited on the poster "is a complex process" and "the omission of Gurira in the top billing was less of a conscious decision than the ramifications of dealing with agents, fees, and movie star demands." However, later in the day Marvel Studios released an updated poster with Gurira\'s name in the top billing. The second trailer was viewed 268 million times in the first 24 hours, becoming the second most viewed trailer in that time period, behind the film\'s first trailer.\n\n\n== Release ==\n\n\n=== Theatrical ===\nAvengers: Endgame had its world premiere at the Los Angeles Convention Center on April 22, 2019. Disney converted the convention center\'s Hall K for the film\'s premiere, working with Dolby and QSC Audio to install a 70 foot (21 metres) screen, Dolby Vision projectors, and a Dolby Atmos sound system. The convention center also held the premiere\'s red carpet arrival and after party. It was released in China, Australia, and other parts of Asia and Europe on April 24, 2019, in the United Kingdom on April 25, in the United States and India on April 26, and Russia on April 29 in IMAX and 3D. It was originally scheduled to be released in the United States on May 3, 2019. As reported by Radio Liberty, it was alleged that the Russian government controversially ordered to postpone the film\'s release in the country in order to promote Russian-produced films.Following the release of the second trailer for Spider-Man: Far From Home on May 6, 2019, Marvel began showing it at the end of Endgame showings until the re-release on June 28th, with a message from that film\'s star Holland telling the audience to stay until the end of the credits to see it.In June 2019, Feige confirmed that Avengers: Endgame would be re-released in theatres with seven minutes of new post-credits footage not included in the original theatrical release, including a Stan Lee tribute, an unfinished deleted scene, and the opening scene of Far From Home, along with a new poster given out at select theaters. The re-release began on June 28 in the United States, with the film returning to an additional 1,040 theaters.\n\n\n=== Home media ===\nAvengers: Endgame will be released in the US on digital download by Walt Disney Studios Home Entertainment on July 30, 2019, and on Ultra HD Blu-ray, Blu-ray, and DVD on August 13. The film will later be released on both Digital HD and Blu-ray on September 2, 2019 in the UK. Streaming will be kept exclusive to Disney+ starting December 11, 2019. The digital and Blu-ray releases will include behind-the-scenes featurettes, audio commentary, deleted scenes, a blooper reel and a bonus feature called Steve and Peggy: One Last Dance which takes a look at Steve Rogers and Peggy Carter’s relationship across the MCU.\n\n\n== Reception ==\n\n\n=== Box office ===\n\nAs of July 4, 2019, Avengers: Endgame has grossed $844.8 million in the United States and Canada, and $1.923 billion in other territories, for a worldwide total of $2.768 billion. It is the highest-grossing film of 2019, as well as the highest-grossing superhero film of all time, highest-grossing film based on a comic book, highest-grossing film released by Walt Disney Studios, second-highest-grossing film of all time worldwide and in the United States and Canada, as well as the fifth-highest-grossing film of all time worldwide when adjusted for inflation.The film had a worldwide opening of $1.2 billion, the biggest of all time and nearly double Infinity War\'s previous record of $640.5 million. It was also the fastest film ever to eclipse the $1 billion and $1.5 billion mark, doing so in just five days and eight days respectively (less than half what it took Infinity War). Deadline Hollywood estimated the film would break-even just five days after its release, which is "unheard of for a major studio tentpole during its opening weekend". The website estimated it would turn a net profit of $600–650 million, accounting for production budgets, P&A, talent participations, and other costs, with box office grosses and ancillary revenues from home media. On May 4, the film\'s earnings at the global box office passed the entire theatrical run of Infinity War and became the fastest film ever to gross $2 billion worldwide, amassing the amount in only 11 days (beating Avatar that did so in 47 days). It also became the fifth film to surpass this threshold (after Avatar, Titanic, Star Wars: The Force Awakens, and Infinity War), and the second film to surpass the threshold of $2.5 billion, doing so in just 20 days, outpacing Avatar\'s record of 72 days.\n\n\n==== Pre-sale records ====\nIn late December 2018, Endgame was named the second-most anticipated 2019 film by IMDb behind Captain Marvel, the most anticipated blockbuster of 2019 according to the ticketing service Fandango, and the most anticipated overall film by Atom Tickets.Due to the high demand when pre-sale tickets became available in the U.S. on April 2, 2019, customers on both Atom Tickets and Fandango experienced long wait times and system delays, while AMC Theatres\' website and app crashed completely for several hours. The same day, Fandango announced the film became its top-selling pre-sale title for the first 24 hours, topping Star Wars: The Force Awakens\' previous record in just six hours. Atom said the film was also the website\'s best selling first-day film (outselling Aquaman by four times), and Regal Cinemas reported that Endgame had sold more tickets in its first eight hours than Infinity War did in its entire first week. The film grossed $120–140 million in pre-sales alone. The day prior to the film\'s release, Fandango announced it was its biggest pre-selling title of all time, beating The Force Awakens, with over 8,000 sold out showtimes across the country.In India, the film sold one million tickets in just one day for the English, Hindi, Tamil, and Telugu language screenings; 18 tickets were sold every second. In China, pre-sale tickets became available on April 12 and sold a record one million tickets in just six hours, outselling Infinity War\'s first 24 hour total within the first hour, and made $114.5 million (RMB 770 million) just from pre-sales.\n\n\n==== United States and Canada ====\n\nOn April 4, industry tracking projected the film would gross $200–250 million domestically during its opening weekend, although some insiders saw those figures as conservative and expected a $260–300 million-debut. By the week of its release, domestic estimates had risen to $260–270 million, with some insiders still suggesting a $300 million debut was possible. The film played in 4,662 theaters, 410 of which were in IMAX; it is the widest release ever, surpassing the record of Despicable Me 3\'s 4,529 theaters. Avengers: Endgame earned $357.1 million in its opening weekend, breaking Infinity War\'s record by nearly $100 million. It also set records for Friday ($157.5 million, including $60 million from Thursday night previews), Saturday ($109.3 million) and Sunday ($90.4 million) totals, as well as was more of a total gross alone than the previous box office high of all films combined ($314 million). The film then made $36.9 million on Monday and $33.1 million on Tuesday, both the third-highest of all time. In its second weekend the film made $147.4 million (the second-best sophomore frame ever) for a 10-day total of $621.3 million. It was the fastest film to ever pass the $600 million milestone, beating The Force Awakens\' 12 days and less than half the 26 days it took Infinity War. The following week, the film grossed $64.8 million, the fourth-best third weekend ever. It also passed the $700 million mark, tying The Force Awakens\' record of 16 days. Endgame was finally dethroned in its fourth weekend by newcomer John Wick: Chapter 3 – Parabellum, however its $29.3 million was enough for it to pass Avatar as the second-highest grossing film ever domestically. It made $17.2 million the following weekend (and a total of $22.3 million over the four-day Memorial Day frame), crossing the $800 million threshold domestically. In its tenth weekend the film was added to 1,040 theaters and made $6.1 million, an increase of 207% from the previous weekend.\n\n\n==== Other territories ====\nInternationally, Endgame was projected to gross around $680 million over its first five days for a global debut of $850–950 million. The film was initially projected to gross $250–280 million in China in its opening weekend, but made a record $107.5 million (RMB 719 million) in the country on its first day, including $28.2 million (RMB 189 million) from midnight, 3 AM and 6 AM screenings, beating The Fate of the Furious\' previous record of $9.1 million. Due to the record-breaking first day, partnered with word of mouth (with a 9.1 on local review aggregator Douban and a 9.3 on ticket website Maoyan), debut projections were increased to over $300 million. Overall, the film made $169 million on the first day from international countries, the highest total of all time. Its largest markets after China were India ($9 million), South Korea ($8.4 million; the largest non-holiday single day gross ever), Australia ($7.1 million), France ($6 million) and Italy ($5.8 million). Like domestically, the film ended up over-performing and debuted to $866.5 million overseas. Not only was it the highest foreign amount ever, but it was single-handedly more than Infinity War\'s global opening of $640 million. Its largest markets, every one of which set the record for best-ever opening in the country, were China ($330.5 million; RMB 2.22 billion), the United Kingdom ($53.8 million), South Korea ($47.4 million), Mexico ($33.1 million), Australia ($30.8 million), Brazil ($26 million), Spain ($13.3 million), Japan ($13 million) and Vietnam ($10 million). It also made $21.6 million over its first four days in Russia after a delay of its premiere that was caused by the Russian government.In its first week, the film\'s top five largest international markets were China ($459.4 million), the United Kingdom ($68.2 million), South Korea ($60.3 million), Mexico ($48.6 million), and India ($40.9 million). A week after its release, it became the highest-grossing foreign film of all time in China and India. In its second weekend the film\'s running total passed $1.569 billion from international markets, passing Titanic as the second-highest film overseas of all time.\n\n\n=== Critical response ===\n\nThe review aggregator website Rotten Tomatoes reported an approval rating of 94% with an average score of 8.3/10, based on 474 reviews. The website\'s critics consensus reads, "Exciting, entertaining, and emotionally impactful, Avengers: Endgame does whatever it takes to deliver a satisfying finale to Marvel\'s epic Infinity Saga." Metacritic, which uses a weighted average, assigned the film a score of 78 out of 100 based on 57 critics, indicating "generally favorable reviews". Audiences polled by CinemaScore gave the film a rare "A+" grade, the third Marvel film to earn the score after The Avengers and Black Panther, while those at PostTrak gave it 5 out of 5 stars and a "definite recommend" of 85%.Writing for NPR, Glen Weldon gave the film a positive review and found the film to be a worthy sequel to its predecessor, stating, "The Russos\' decision to stick close to the experiences of the remaining Avengers proves a rewarding one, as they\'ve expressly constructed the film as an extended victory lap for the Marvel Cinematic Universe writ large. Got a favorite character from any Marvel movie over the past decade, no matter how obscure? Prepare to get serviced, fan." Peter Travers in his review for Rolling Stone magazine gave the film 4 out of 5 stars, saying, "You don’t have to make jokes about the clichéd time travel plot — the film is ready, willing and able to make its own, with Back to the Future coming in for a serious ribbing." Peter Debruge of Variety wrote, "After the must-see showdown that was Infinity War, the Russo brothers deliver a more fan-facing three-hour followup, rewarding loyalty to Marvel Cinematic Universe. J.R. Kinnard of PopMatters wrote, "Big budget action filmmaking doesn\'t get much better than this." Todd McCarthy of The Hollywood Reporter said, "[W]hat comes across most strongly here, oddly enough for an effects-driven comic-book-derived film, is the character acting, especially from Downey, Ruffalo, Evans, Hemsworth, Brolin and Paul Rudd". Richard Roeper, writing for the Chicago Sun-Times, gave the film four stars and called it "the undisputed champion [of the MCU] when it comes to emotional punch". Roeper went on to praise the "funny, well-paced, smart, expertly rendered screenplay by Christopher Markus and Stephen McFeely, crisp direction from Anthony Russo and Joe Russo..., and the universally stellar performances [of the cast]."The New York Times reviewer A. O. Scott gave the film a positive though guarded review, stating, "Endgame is a monument to adequacy, a fitting capstone to an enterprise that figured out how to be good enough for enough people enough of the time. Not that it’s really over, of course: Disney and Marvel are still working out new wrinkles in the time-money continuum. But the Russos do provide the sense of an ending, a chance to appreciate what has been done before the timelines reset and we all get back to work." Justin Chang of the Los Angeles Times wrote that "Avengers: Endgame achieves and earns its climactic surge of feeling, even as it falls just short of real catharsis".Some have noted the film as a notable improvement over its predecessor, Avengers: Infinity War, such as Brian Tallerico of RogerEbert.com, who stated that Endgame is "a more patient, focused film [than Infinity War], even as its plot draws in elements of a dozen other movies." Matt Zoller Seitz, also of RogerEbert.com, gave the film a positive assessment as compared with Infinity War, which he considered "too crowded, too rushed and yet too long". Seitz stated that Endgame is "a heartfelt and satisfying experience", along with being a "surprisingly relaxed, character-driven, self-aware yet sincere comedy [for] two-thirds of [the film]. Much of the script suggests a laid-back Richard Linklater movie with superheroes".Richard Brody, writing for The New Yorker, was more critical of the film, opining that the good acting was not matched by comparable skill from the directors. He said, "The Russos have peculiarly little sense of visual pleasure, little sense of beauty, little sense of metaphor, little aptitude for texture or composition; their spectacular conceit is purely one of scale, which is why their finest moments are quiet and dramatic ones". Anthony Lane of The New Yorker gave the film a compromising review, finding it to be overdeveloped and overwrought, stating, "The one thing you do need to know about Avengers: Endgame is that it runs for a little over three hours, and that you can easily duck out during the middle hour, do some shopping, and slip back into your seat for the climax. You won\'t have missed a thing."\n\n\n=== Accolades ===\n\n\n== Future ==\n\nIn May 2018, Disney CEO Bob Iger said of Marvel\'s plans beyond Endgame, "I\'m guessing we will try our hand at what I\'ll call a new franchise beyond Avengers, but that doesn\'t necessarily mean you won\'t see more Avengers down the road. We just haven\'t made any announcements about that." Iger added, "Given the popularity of the characters and given the popularity of the franchise, I don\'t think people should conclude there will never be another Avengers movie." Shortly after the film\'s premiere, Anthony Russo stated that he and Joe Russo did not "have any plans for now to make any more Marvel movies" after Endgame, but that they were not opposed to returning to the MCU in the future due to their positive relationship with Marvel Studios.\n\n\n== Notes ==\n\n\n== References ==\n\n\n== External links ==\n\nOfficial website \nAvengers: Endgame on IMDb''',
        "Who was casted as Iron Man ?")
for row in result:
    print(row['text'])

UnparsedFlagAccessError: ignored